In [1]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 847.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup

def scrape_wiki_main_page(url: str) -> str:
    """
    Scrapes news content from wikipedia's main page.

    Args:
        url (str): The URL of the wikipedia page to scrape.

    Returns:
        str: The text content of the webpage.
             Returns None if there is an error during the process.
    """

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            content = soup.get_text()
            return content
            content = content[content.find("In the news"):content.find("Ongoing")]
            return content

        else:
            print("Failed to retrieve webpage. Status code:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None


print(scrape_wiki_main_page('https://en.wikipedia.org/wiki/Main_Page'))





Wikipedia, the free encyclopedia






































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search






















Appearance
















Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk



























Main Page










Main PageTalk





English

















ReadView sourceView history







Tools





Tools
move to sidebar
hide



		Actions
	


ReadView sourceView history





		General
	


What links hereRelated changesUpload fileSpecial pagesPermanent linkPage informationCite this pageGet shortened URLDownload QR codeWikidata item





		Print/export
	


Download as PDFPrintable version





		In oth

In [3]:
import os
from google.colab import userdata
from autogen import ConversableAgent

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="""You are a helpful AI news bot.
    You can pull content from wikipedia's main page.
    URL: https://en.wikipedia.org/wiki/Main_Page. Once pulled, only share the top 3 stories.
    Return 'TERMINATE' when the task is done.""",
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="user_proxy",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Register the tool's information with the assistant agent.
assistant.register_for_llm(
    name="scrape_wiki_main_page",
    description="A tool for scraping today's news from wikipedia"
)(scrape_wiki_main_page)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(
    name="scrape_wiki_main_page"
)(scrape_wiki_main_page)

[autogen.oai.client: 09-02 15:59:00] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-02 15:59:00] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


<function __main__.scrape_wiki_main_page(url: str) -> str>

In [5]:
from autogen import Cache

with Cache.disk() as cache:
  chat_result = user_proxy.initiate_chat(
      assistant,
      message="What's hot in today's news?",
      max_turns=5
  )

user_proxy (to assistant):

What's hot in today's news?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_q1k5jK6f0MxbRYZFOvn9Mica): scrape_wiki_main_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Main_Page"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_wiki_main_page...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_q1k5jK6f0MxbRYZFOvn9Mica) *****




Wikipedia, the free encyclopedia






































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent ch

In [6]:
from autogen import Cache

with Cache.disk() as cache:
  chat_result = user_proxy.initiate_chat(
      assistant,
      message="What's hot in today's news?",
      max_turns=2
  )

print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

chat_result.summary

user_proxy (to assistant):

What's hot in today's news?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_q1k5jK6f0MxbRYZFOvn9Mica): scrape_wiki_main_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Main_Page"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_wiki_main_page...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_q1k5jK6f0MxbRYZFOvn9Mica) *****




Wikipedia, the free encyclopedia






































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent ch

'Here are the top 3 news stories for today:\n\n1. The Summer Paralympics open in Paris, France.\n2. More than four hundred people are killed in an Islamist militant attack in Barsalogho Department, Burkina Faso.\n3. The Islamic State claims responsibility for a mass stabbing that killed three people at a festival in Solingen, Germany.\n\nWould you like to know more details about any of these news stories?'